In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# delete this cell if working on Pycharm
!pip install Bio

In [15]:
from Bio.PDB import *
import numpy as np
import os
from tqdm import tqdm

In [16]:
NB_MAX_LENGTH = 140
AA_DICT = {"A": 0, "C": 1, "D": 2, "E": 3, "F": 4, "G": 5, "H": 6, "I": 7, "K": 8, "L": 9, "M": 10, "N": 11,
           "P": 12, "Q": 13, "R": 14, "S": 15, "T": 16, "W": 17, "Y": 18, "V": 19, "X": 20, "-": 21}
FEATURE_NUM = len(AA_DICT)
BACKBONE_ATOMS = ["N", "CA", "C", "O", "CB"]
OUTPUT_SIZE = len(BACKBONE_ATOMS) * 3
NB_CHAIN_ID = "H"


In [17]:
def get_seq_aa(pdb_file, chain_id):
    """
    returns the sequence (String) and a list of all the aa residue objects of the given protein chain.
    :param pdb_file: path to a pdb file
    :param chain_id: chain letter (char)
    :return: sequence, [aa objects]
    """
    # load model
    chain = PDBParser(QUIET=True).get_structure(pdb_file, pdb_file)[0][chain_id]

    aa_residues = []
    seq = ""

    for residue in chain.get_residues():
        aa = residue.get_resname()
        if not is_aa(aa) or not residue.has_id('CA'): # Not amino acid
            continue
        elif aa == "UNK":  # unkown amino acid
            seq += "X"
        else:
            seq += Polypeptide.index_to_one(Polypeptide.three_to_index(residue.get_resname()))
        aa_residues.append(residue)

    return seq, aa_residues

In [18]:
def generate_input(pdb_file): # TODO: implement this!
    """
    receives a pdb file and returns its sequence in a one-hot encoding matrix (each row is an aa in the sequence, and
    each column represents a different aa out of the 20 aa + 2 special columns).
    :param pdb_file: path to a pdb file (nanobody, heavy chain has id 'H')
    :return: numpy array of shape (NB_MAX_LENGTH, FEATURE_NUM)
    """

    # get seq and aa residues
    seq, _ = get_seq_aa(pdb_file, NB_CHAIN_ID)
    encoded_one_hots = np.zeros((NB_MAX_LENGTH, FEATURE_NUM))


    for i, aa in enumerate(seq):
        #Todo check if this is correct
        if aa not in AA_DICT:
            encoded_one_hots[i][AA_DICT['X']] = 1
        else:
            encoded_one_hots[i][AA_DICT[aa]] = 1

    if len(seq) < NB_MAX_LENGTH:
        encoded_one_hots[len(seq):, AA_DICT['-']] = 1

    return encoded_one_hots

In [19]:
def generate_label(pdb_file):  # TODO: implement this!
    """
    receives a pdb file and returns its backbone + CB coordinates.
    :param pdb_file: path to a pdb file (nanobody, heavy chain has id 'H') already alingned to a reference nanobody.
    :return: numpy array of shape (CDR_MAX_LENGTH, OUTPUT_SIZE).
    """
    # get seq and aa residues
    seq, aa_residues = get_seq_aa(pdb_file, NB_CHAIN_ID)

    coorrdinates_arr = np.zeros((NB_MAX_LENGTH,OUTPUT_SIZE ))

    for i, res in enumerate(aa_residues):
        n_atom_coords = res['N'].get_coord()
        ca_atom_coords = res["CA"].get_coord()
        c_atom_coords = res["C"].get_coord()
        o_atom_coords = res["O"].get_coord()

        if res.get_resname() != "GLY" and "CB" in res:
            cb_atom_coords = res["CB"].get_coord()
        else:
            cb_atom_coords = np.zeros(3)

        coorrdinates_arr[i, 0:3] = n_atom_coords
        coorrdinates_arr[i, 3:6] = ca_atom_coords
        coorrdinates_arr[i, 6:9] = c_atom_coords
        coorrdinates_arr[i, 9:12] = o_atom_coords
        coorrdinates_arr[i, 12:15] = cb_atom_coords

    return coorrdinates_arr


In [20]:
def matrix_to_pd(seq, coord_matrix, pdb_name):
    """
    Receives a sequence (String) and the output matrix of the neural network (coord_matrix, numpy array)
    and creates from them a PDB file named pdb_name.pdb.
    :param seq: protein sequence (String), with no padding
    :param coord_matrix: output np array of the nanobody neural network, shape = (NB_MAX_LENGTH, OUTPUT_SIZE)
    :param pdb_name: name of the output PDB file (String)
    """
    ATOM_LINE = "ATOM{}{}  {}{}{} {}{}{}{}{:.3f}{}{:.3f}{}{:.3f}  1.00{}{:.2f}           {}\n"
    END_LINE = "END\n"
    k = 1
    one_to_three_dict = {
        "A": "ALA", "R": "ARG", "N": "ASN", "D": "ASP",
        "C": "CYS", "E": "GLU", "Q": "GLN", "G": "GLY",
        "H": "HIS", "I": "ILE", "L": "LEU", "K": "LYS",
        "M": "MET", "F": "PHE", "P": "PRO", "S": "SER",
        "T": "THR", "W": "TRP", "Y": "TYR", "V": "VAL",
        "-": "GLY", "X": "GLY"
    }
    with open(f"{pdb_name}.pdb", "w") as pdb_file:
        for i, aa in enumerate(seq):
            third_space = (4 - len(str(i))) * " "
            for j, atom in enumerate(BACKBONE_ATOMS):
                if not (aa == "G" and atom == "CB"):  # GLY lacks CB atom
                    x, y, z = coord_matrix[i][3*j], coord_matrix[i][3*j+1], coord_matrix[i][3*j+2]
                    b_factor = 0.00
                    first_space = (7 - len(str(k))) * " "
                    second_space = (4 - len(atom)) * " "
                    forth_space = (12 - len("{:.3f}".format(x))) * " "
                    fifth_space = (8 - len("{:.3f}".format(y))) * " "
                    sixth_space = (8 - len("{:.3f}".format(z))) * " "
                    seventh_space = (6 - len("{:.2f}".format(b_factor))) * " "

                    pdb_file.write(ATOM_LINE.format(first_space, k, atom, second_space,
                                                    one_to_three_dict.get(aa, "GLY") , "H", third_space,
                                                    i, forth_space, x, fifth_space, y,
                                                    sixth_space, z, seventh_space,
                                                    b_factor, atom[0]))
                    k += 1

        pdb_file.write(END_LINE)

In [ ]:
# def matrix_to_pdb(seq, coord_matrix, pdb_name):
#     """
#     Receives a sequence (String) and the output matrix of the neural network (coord_matrix, numpy array)
#     and creates from them a PDB file named pdb_name.pdb.
#     :param seq: protein sequence (String), with no padding
#     :param coord_matrix: output np array of the nanobody neural network, shape = (NB_MAX_LENGTH, OUTPUT_SIZE)
#     :param pdb_name: name of the output PDB file (String)
#     """
#     ATOM_LINE = "ATOM{}{}  {}{}{} {}{}{}{}{:.3f}{}{:.3f}{}{:.3f}  1.00{}{:.2f}           {}\n"
#     END_LINE = "END\n"
#     k = 1
#     with open(f"{pdb_name}.pdb", "w") as pdb_file:
#         for i, aa in enumerate(seq):
#             third_space = (4 - len(str(i))) * " "
#             for j, atom in enumerate(BACKBONE_ATOMS):
#                 if not (aa == "G" and atom == "CB"):  # GLY lacks CB atom
#                     x, y, z = coord_matrix[i][3*j], coord_matrix[i][3*j+1], coord_matrix[i][3*j+2]
#                     b_factor = 0.00
#                     first_space = (7 - len(str(k))) * " "
#                     second_space = (4 - len(atom)) * " "
#                     forth_space = (12 - len("{:.3f}".format(x))) * " "
#                     fifth_space = (8 - len("{:.3f}".format(y))) * " "
#                     sixth_space = (8 - len("{:.3f}".format(z))) * " "
#                     seventh_space = (6 - len("{:.2f}".format(b_factor))) * " "

#                     pdb_file.write(ATOM_LINE.format(first_space, k, atom, second_space, Polypeptide.one_to_three(aa) , "H", third_space,
#                                                     i, forth_space, x, fifth_space, y, sixth_space, z, seventh_space,
#                                                     b_factor, atom[0]))
#                     k += 1

#         pdb_file.write(END_LINE)

In [ ]:
if __name__ == '__main__':

   #  you can make all the data for the network in this section.
   # you can save the matrices to your drive and load them in your google colab file later.


    input_matrix = []
    labels_matrix = []
    data_path = "/content/drive/MyDrive/bio_ex3_new/Ex4Data"  # TODO: change path if needed

    for pdb in tqdm(os.listdir(data_path)):

        nb_one_hot = generate_input(os.path.join(data_path, pdb))
        nb_xyz = generate_label(os.path.join(data_path, pdb))

        input_matrix.append(nb_one_hot)
        labels_matrix.append(nb_xyz)

    save_path = "/content/drive/MyDrive/bio_ex3_new"  # TODO: change path if needed

    np.save(f"{save_path}/train_input.npy", np.array(input_matrix))
    np.save(f"{save_path}/train_labels.npy", np.array(labels_matrix))

    print(f"Number of samples: {len(input_matrix)}")

